In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("merge_df.csv")

In [ ]:
df

,Unnamed: 0,File_path,Articles,Summaries
0,0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...
1,1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ..."
2,2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
3,3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...
4,4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g..."
...,...,...,...,...
5444,5444,accidents,HONG KONG — Hundreds of pilot whales that s...,more than 500 rescuers tried frantically to se...
5445,5445,sports,"NICE, France — Rivère accepts the complim...",Signing balotelli was not just a way to garner...
5446,5446,business,FRANKFURT — Germans who never really warmed...,Although there was no evidence of that the bun...
5447,5447,sports,Charles Oakley has strong feelings about compe...,He questioned why any n. b. a. free agent woul...


In [ ]:
df.shape


(5449, 4)

In [ ]:
df = df.drop(columns="Unnamed: 0")


In [ ]:
df

,File_path,Articles,Summaries
0,politics,Budget to set scene for election..Gordon Brown...,- Increase in the stamp duty threshold from £6...
1,politics,Army chiefs in regiments decision..Military ch...,"""They are very much not for the good and will ..."
2,politics,Howard denies split over ID cards..Michael How...,Michael Howard has denied his shadow cabinet w...
3,politics,Observers to monitor UK election..Ministers wi...,The report said individual registration should...
4,politics,Kilroy names election seat target..Ex-chat sho...,"UKIP's leader, Roger Knapman, has said he is g..."
...,...,...,...
5444,accidents,HONG KONG — Hundreds of pilot whales that s...,more than 500 rescuers tried frantically to se...
5445,sports,"NICE, France — Rivère accepts the complim...",Signing balotelli was not just a way to garner...
5446,business,FRANKFURT — Germans who never really warmed...,Although there was no evidence of that the bun...
5447,sports,Charles Oakley has strong feelings about compe...,He questioned why any n. b. a. free agent woul...


In [ ]:
df['File_path'].unique()


array(['politics', 'sport', 'tech', 'entertainment', 'business', 'crime',
       'technology', 'art', 'science', 'health', 'architecture',
       'lifestyle', 'sports', 'law', 'accidents', 'environment '],
      dtype=object)

In [ ]:
filtered_df = df[df['File_path'].isin(['sports', 'sport'])]
filtered_df = df[df['File_path'] == 'sport']

In [ ]:
filtered_df

,File_path,Articles,Summaries
417,sport,Hodges announces rugby retirement..Scarlets an...,"The 36-year-old, who has 54 caps, was Llanelli..."
418,sport,Bomb threat at Bernabeu stadium..Spectators we...,Spectators were evacuated from Real Madrid's B...
419,sport,Parmar ruled out of Davis Cup tie..A knee inju...,The unheralded Sherwood was the surprise inclu...
420,sport,Benitez issues warning to Gerrard..Liverpool m...,"Benitez responded: ""I spoke to Steven and said..."
421,sport,Officials respond in court row..Australian ten...,"Hewitt said he had had a ""gutful"" of trying to..."
...,...,...,...
3147,sport,Big guns ease through in San Jose\n\nTop-seede...,Top-seeded Americans Andy Roddick and Andre Ag...
3148,sport,Almagro continues Spanish surge\n\nUnseeded Ni...,"Nadal, playing in the outdoor clay event for t..."
3149,sport,Melzer shocks Agassi in San Jose\n\nSecond see...,Second seed Andre Agassi suffered a comprehens...
3150,sport,Mirza makes Indian tennis history\n\nTeenager ...,Teenager Sania Mirza completed a superb week a...


In [ ]:
!pip install rouge-score

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset, DataLoader,Dataset,random_split
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.cuda.amp import autocast, GradScaler
from torch.nn.utils import clip_grad_norm_
from rouge_score import rouge_scorer


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
max_length = 1024

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def tokenize_text(text):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=382, truncation=True, padding='max_length', return_attention_mask=True)
    return inputs.to(device)  # Move the tokenized inputs to the GPU

def tokenize_summary(text):
    inputs = tokenizer.encode(text, return_tensors="pt", max_length=155, truncation=True, padding='max_length', return_attention_mask=True)
    return inputs.to(device)  # Move the tokenized summaries to the GPU


filtered_df['TokenizedText'] = filtered_df['Articles'].apply(tokenize_text)
filtered_df['TokenizedSummary'] = filtered_df['Summaries'].apply(tokenize_summary)

<ipython-input-13-6ecda2fb25da>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['TokenizedText'] = filtered_df['Articles'].apply(tokenize_text)
<ipython-input-13-6ecda2fb25da>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['TokenizedSummary'] = filtered_df['Summaries'].apply(tokenize_summary)


In [ ]:
train_df, test_df = train_test_split(filtered_df, test_size=0.2, random_state=42)

In [ ]:
X_train = torch.stack([seq.squeeze() for seq in train_df['TokenizedText']])
Y_train = torch.stack([seq.squeeze() for seq in train_df['TokenizedSummary']])
X_test = torch.stack([seq.squeeze() for seq in test_df['TokenizedText']])
Y_test = torch.stack([seq.squeeze() for seq in test_df['TokenizedSummary']])

In [ ]:
train_dataset = TensorDataset(X_train, Y_train)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataset = TensorDataset(X_test, Y_test)
test_dataloader = DataLoader(test_dataset, batch_size=4)

In [ ]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')

In [ ]:
from torch.cuda.amp import GradScaler

scaler = GradScaler()


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the GPU
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=50, num_training_steps=len(train_dataloader) * 10)  # Add learning rate scheduler
early_stopping_rounds = 2
best_rouge_score = -1
current_round = 0


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
accumulation_steps = 20  # You can adjust this number

def train(model, dataloader, optimizer, scheduler):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()

    for step, batch in enumerate(tqdm(dataloader, desc="Training")):
        inputs = batch[0].to(device)  # Move the input batch to the GPU
        attention_mask = (inputs != 0).float().to(device)  # Create attention mask
        targets = batch[1].to(device)  # Move the target batch to the GPU

        with autocast():
            outputs = model(input_ids=inputs, attention_mask=attention_mask, decoder_input_ids=targets, labels=targets)
            loss = outputs.loss

        # Perform gradient accumulation
        loss = loss / accumulation_steps
        scaler.scale(loss).backward()

        if (step + 1) % accumulation_steps == 0:
            # Update gradients and optimizer once every accumulation_steps
            clip_grad_norm_(model.parameters(), max_norm=1.0)  # Optional gradient clipping
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        total_loss += loss.item()

    return total_loss / len(dataloader)



In [ ]:
for epoch in range(3):
    train_loss = train(model, train_dataloader, optimizer, scheduler)
    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}")

Training: 100%|██████████| 204/204 [00:28<00:00,  7.19it/s]


Epoch 1, Train Loss: 0.5599


Training: 100%|██████████| 204/204 [00:25<00:00,  8.02it/s]


Epoch 2, Train Loss: 0.5613


Training: 100%|██████████| 204/204 [00:25<00:00,  8.10it/s]

Epoch 3, Train Loss: 0.5607


In [ ]:
def evaluate1(model, dataloader):
    model.eval()

    test_articles = []
    actual_summaries = []
    predicted_summaries = []
    rouge1_precision_scores = []

    scorer = rouge_scorer.RougeScorer(['rouge1'])

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating Test"):
            inputs = batch[0].to(device)
            attention_mask = (inputs != 0).float().to(device)
            targets = batch[1].to(device)
            outputs = model.generate(input_ids=inputs, attention_mask=attention_mask, max_length=155, num_beams=30, length_penalty=3.0, early_stopping=True)

            for output, target, input_text in zip(outputs, targets, inputs):
                # Calculate ROUGE-1 precision for each sample
                output_text = tokenizer.decode(output, skip_special_tokens=True)
                target_text = tokenizer.decode(target, skip_special_tokens=True)
                rouge_scores = scorer.score(output_text, target_text)
                rouge1_precision_scores.append(rouge_scores['rouge1'].precision)

                # Append tokenized text, actual summaries, and predicted summaries
                test_articles.append(tokenizer.decode(input_text, skip_special_tokens=True))
                actual_summaries.append(target_text)
                predicted_summaries.append(output_text)

    return test_articles, actual_summaries, predicted_summaries, rouge1_precision_scores

In [ ]:
def evaluate2(model, dataloader):
    model.eval()

    test_articles = []
    actual_summaries = []
    predicted_summaries = []
    rouge2_precision_scores = []

    scorer = rouge_scorer.RougeScorer(['rouge2'])

    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating Test"):
            inputs = batch[0].to(device)
            attention_mask = (inputs != 0).float().to(device)
            targets = batch[1].to(device)
            outputs = model.generate(input_ids=inputs, attention_mask=attention_mask, max_length=155, num_beams=30, length_penalty=3.0, early_stopping=True)

            for output, target, input_text in zip(outputs, targets, inputs):
                # Calculate ROUGE-2 precision for each sample
                output_text = tokenizer.decode(output, skip_special_tokens=True)
                target_text = tokenizer.decode(target, skip_special_tokens=True)
                rouge_scores = scorer.score(output_text, target_text)
                rouge2_precision_scores.append(rouge_scores['rouge2'].precision)

                # Append tokenized text, actual summaries, and predicted summaries
                test_articles.append(tokenizer.decode(input_text, skip_special_tokens=True))
                actual_summaries.append(target_text)
                predicted_summaries.append(output_text)

    return test_articles, actual_summaries, predicted_summaries, rouge2_precision_scores

In [ ]:
# Evaluate the model
test_articles, actual_summaries, predicted_summaries, rouge1_precision_scores = evaluate1(model, test_dataloader)

# Create a dictionary with the extracted data
data = {
    'Article': test_articles,
    'Actual Summary': actual_summaries,
    'Predicted Summary': predicted_summaries,
    'ROUGE-1 Precision': rouge1_precision_scores,
}

# Create a Pandas DataFrame from the dictionary
results_rouge1_df = pd.DataFrame(data)


# Display the DataFrame
pd.set_option('display.max_colwidth', None)
results_rouge1_df.head(5)

Evaluating Test:   4%|▍         | 2/52 [00:23<09:53, 11.88s/it]


KeyboardInterrupt: 

In [ ]:
results_rouge1_df.to_excel('results_rouge1.xlsx')

In [ ]:
test_articles, actual_summaries, predicted_summaries, rouge2_precision_scores = evaluate2(model, test_dataloader)

# Create a dictionary with the extracted data
data = {
    'Article': test_articles,
    'Actual Summary': actual_summaries,
    'Predicted Summary': predicted_summaries,
    'ROUGE-2 Precision': rouge2_precision_scores,
}

# Create a Pandas DataFrame from the dictionary
results_rouge2_df = pd.DataFrame(data)

results_rouge2_df.head(5)

In [ ]:
results_rouge2_df.to_excel('results_rouge22.xlsx')

In [ ]:
model.save_pretrained("pre_train_sum_batch_4")

In [ ]:
abc="im Clijsters has denied reports that she has pulled out of January's Australian Open because of her persistent wrist injury.Dokic has not played in the Australian Open since 2001 when she lost in the first round.Four-time "

In [ ]:
input_ids = input_ids.to(device)


In [ ]:
summary_ids = model.generate(input_ids=input_ids, max_length=1024, num_beams=17, length_penalty=2.0, early_stopping=False)
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
generated_summary

"im Clijsters has denied reports that she has pulled out of January's Australian Open because of her persistent wrist injury.Dokic has not played in the Australian Open since 2001 when she lost in the first round.Four-time "

In [ ]:
import string

In [ ]:
def clean_text(text):
    # Remove punctuation and convert to lowercase
    translator = str.maketrans("", "", string.punctuation)
    cleaned_text = text.translate(translator).lower()
    return cleaned_text

In [ ]:
def rouge_1_evaluation(reference, generated):
    reference = clean_text(reference)
    generated = clean_text(generated)

    reference_tokens = reference.split()
    generated_tokens = generated.split()

    # Calculate ROUGE-1 recall
    overlapping_tokens = set(reference_tokens) & set(generated_tokens)
    recall = len(overlapping_tokens) / len(reference_tokens) if len(reference_tokens) > 0 else 0.0

    return recall

In [ ]:
rouge_1_recall = rouge_1_evaluation(abc, generated_summary)
print(f"ROUGE-1 Recall: {rouge_1_recall}")

ROUGE-1 Recall: 0.7777777777777778


In [ ]:
def rouge_2_evaluation(reference, generated):
    reference = clean_text(reference)
    generated = clean_text(generated)

    reference_bigrams = list(zip(reference.split()[:-1], reference.split()[1:]))
    generated_bigrams = list(zip(generated.split()[:-1], generated.split()[1:]))

    # Calculate ROUGE-2 recall
    overlapping_bigrams = set(reference_bigrams) & set(generated_bigrams)
    recall = len(overlapping_bigrams) / len(reference_bigrams) if len(reference_bigrams) > 0 else 0.0

    return recall

In [ ]:
rouge_2_recall = rouge_2_evaluation(abc, generated_summary)
print(f"ROUGE-2 Recall: {rouge_2_recall}")

ROUGE-2 Recall: 0.9428571428571428
